# Preliminaries

We shall be using pennylane from now on, unless otherwise noted. Installation of pennylane is very similar to qiskit. See [here](https://pennylane.ai/install.html). You might want to check cuQuantum acceleration [lightning.gpu](https://github.com/PennyLaneAI/pennylane-lightning-gpu)

# How to embed data into a quantum state


 In classical computing problems, data is classical. Does it make sense for quantum computers to deal with classical data? The short answer is yes! In this how-to, the first few steps of how to encode classical data into a quantum state is presented.

## Different embedding types

To encode your classical data into a quantum state, you first have to find out what type of classical data you have. We will distinguish between three different types of data in $N-$dimensions
1. Discrete data, represented as binary $\mathbf{b}\in{0,1}^N$ or integer values $\mathbf{k}\in\mathbb{Z}^N$
2. Real continuous data, represented as floating-point values $\mathbf{k}\in\mathbb{R}^N$
3. Complex continuous data, represented as complex values $\mathbf{\alpha}\in\mathbb{C}^2^N$

Keeping the subset relations $\{0, 1\}\subset\mathbb{Z}\subset\mathbb{R}\subset\mathbb{C}$  in mind, one could always choose to interpret the data in the domain $\mathcal{D}$ to be in the superset $\mathcal{D}^{\prime}\supset \mathcal{D}$

## 1. Discrete data, represented as binary or integer values

A suitable encoding for binary data is the so-called `BasisEmbedding`. The BasisEmbedding class interprets a binary string as a qubit basis state with the following mapping:

$$
\mathbf{b}=(b_{0}, \ldots, b_{N-1}) \mapsto |b_{0}, \ldots, b_{N-1}\rangle.
$$

See below for a simple example of the `BasisEmbedding` used to initialize three qubits.

In [4]:
import pennylane as qml

N = 3
wires = range(N)
dev = qml.device("default.qubit", wires)

@qml.qnode(dev)
def circuit(b):
    qml.BasisEmbedding(b, wires)
    return qml.state()

circuit([1, 1, 1])

tensor([0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 1.+0.j], requires_grad=True)

As expected, the result corresponds to the state |111⟩=|1⟩⊗|1⟩⊗|1⟩. Representing the |1⟩

state as the second standard basis vector and the tensor product as the Kronecker product, we can confirm the result with a quick calculation.

$$
\left| 1 \right \rangle \otimes \left| 1 \right  \rangle \otimes \left| 1 \right \rangle = \begin{bmatrix}0 \\
1\end{bmatrix}
\otimes
\begin{bmatrix}
0 \\
1\end{bmatrix}
\otimes
\begin{bmatrix}
0 \\
1\end{bmatrix}
=
\begin{bmatrix}0 & 0 & 0 & 0 & 0 & 0 & 0 & 1\end{bmatrix}^{\top}
$$

You can also just pass an integer value to the basis embedding function and it will automatically convert it into its binary representation. We can perform a quick sanity check of this functionality by running

In [5]:
 print(circuit(7))

[0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 1.+0.j]


which is the same state vector we saw before. Unsurprisingly, the binary label corresponding to this state vector is also consistent with the binary representation of the integer seven.

## Send it after class
Embed integer value $32$ using `basisembedding`

## 2. Continuous data, represented as floating-point values

The simplest type of encoding for floating-point data is called `AngleEmbedding`. This type of embedding encodes a single floating-point value $ x\in \mathbb{R} $ into a quantum state with the mapping
$$
x \mapsto R_{k}(x)|0\rangle = e^{-i x\sigma_{k}/2}|0\rangle,
$$
where $k\in\{x, y, z\}$ is the axis of rotation in the Bloch sphere. The default axis of rotation is set to $k=x$ in the `AngleEmbedding` class. You may also choose to set it to $k=y$, but make sure to avoid $k=z$. The latter case is not useful because every $x$ will be mapped to the $|0\rangle$ state; the encoded value will be lost. Note that you can also input a tensor-like object and encode each component as a qubit. Examine the code snippet below to see how to encode a classical floating-point value as a quantum state!

In [6]:
import pennylane as qml
from pennylane import numpy as np

N = 3
wires = range(3)
dev = qml.device("default.qubit", wires)

@qml.qnode(dev)
def circuit(val_list):
    qml.AngleEmbedding(val_list, wires)
    return [qml.expval(qml.PauliZ(w)) for w in wires]
circuit([0.0, np.pi / 2, np.pi])

tensor([ 1.00000000e+00,  2.22044605e-16, -1.00000000e+00], requires_grad=True)

Keep in mind that Pauli rotations are $2\pi$-periodic up to a global phase, meaning that you should normalize your data to be in $\Omega:=[0, \pi)\subset \mathbb{R}$ if possible. This can be helpful in order to avoid encoding two different values as the same quantum state. While the `AngleEmbedding` allows you to encode a lot of information in a single qubit, this comes at the cost of a difficult construction process.

## 3. Continuous data, represented as complex values

Next is the `AmplitudeEmbedding`. As the name suggests, an array of values can be used as the amplitudes of the state with the mapping
$$
\boldsymbol{\alpha}=(\alpha_0, \ldots, \alpha_{2^N-1})\mapsto \sum_{k=0}^{2^N-1}\alpha_{k}|k\rangle
$$
and can be implemented with the following code.

In [7]:
import pennylane as qml

N = 3
wires = range(N)
dev = qml.device("default.qubit", wires)

@qml.qnode(dev)
def circuit(features):
    qml.AmplitudeEmbedding(features, wires)
    return qml.state()
circuit([0.625, 0.0, 0.0, 0.0, 0.625j, 0.375, 0.25, 0.125])

tensor([0.625+0.j   , 0.   +0.j   , 0.   +0.j   , 0.   +0.j   ,
        0.   +0.625j, 0.375+0.j   , 0.25 +0.j   , 0.125+0.j   ], requires_grad=True)

Here, the values were chosen to be normalized, i.e. $\lVert\boldsymbol{\alpha}\rVert=1$. Note that one can use unnormalized data by setting the normalize parameter of the `AmplitudeEmbedding` class to True.

## Templates

PennyLane provides a growing library of pre-coded templates of common variational circuit architectures that can be used to easily build, evaluate, and train more complex models. In the literature, such architectures are commonly known as an ansatz. Templates can be used to embed data into quantum states, to define trainable layers of quantum gates, to prepare quantum states as the first operation in a circuit, or simply as general subroutines that a circuit is built from.

### Embedding templates

Embeddings encode input features into the quantum state of the circuit. Hence, they usually take a data sample such as a feature vector as an argument. Embeddings can also depend on trainable parameters, and they may be constructed from repeated layers.

You can reach Embedding templates [here](https://docs.pennylane.ai/en/stable/introduction/templates.html)


## Send it after class 2

An interesting embedding approach is using the Quantum Approximate Optimization Algorithm  (QAOA) for feature embedding. The template is [here](https://docs.pennylane.ai/en/stable/code/api/pennylane.QAOAEmbedding.html)